<a href="https://colab.research.google.com/github/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/blob/master/notebooks/02_finetune_v1_malaysian_mistral_7b_32k_instructions_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets bitsandbytes peft trl wandb -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 88.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.1/108.1 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import json
import argparse
from random import randint

import wandb
from huggingface_hub import notebook_login

import pandas as pd
from datasets import Dataset, load_dataset

import torch
import bitsandbytes as bnb
from peft import LoraConfig, PeftModel
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from transformers.trainer_utils import get_last_checkpoint
from transformers import AutoModelForCausalLM, AutoTokenizer, \
                         BitsAndBytesConfig, TrainingArguments, \
                         logging, pipeline

In [ ]:
os.environ["WANDB_PROJECT"]="finetune-malaysian-mistral-llmasajudge-v1"

In [ ]:
notebook_login()

In [ ]:
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [1]:
!nvidia-smi

Fri Oct 18 08:45:28 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0              47W / 400W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

### 1.0 Load dataset and prepare the prompt input according to the Mistral format

The [mesolitica/malaysian-mistral-7b-32k-instructions-v4](https://huggingface.co/mesolitica/malaysian-mistral-7b-32k-instructions-v4) is a conversational chat model meaning we can chat with it using the following prompt:

> \<s> [INST] User Instruction 1 [/INST] Model answer 1\</s> [INST] User instruction 2 [/INST]

For instruction fine-tuning, it is quite common to have two columns inside the dataset: one for the prompt & the other for the response.

***NOTE:** This dataset was finetuned using the t5 generated version (instead of the later OpenAI generated version) on the boolq-malay dataset. This can be found here: https://github.com/wanadzhar913/aitinkerers-hackathon-supa-team-werecooked/tree/master/datasets/t5_generated

In [ ]:
dataset_train = load_dataset("wanadzhar913/boolq-malay", split="train")

print(f"Train dataset size: {len(dataset_train)}")

boolq-train.jsonl:   0%|          | 0.00/13.3M [00:00<?, ?B/s]

boolq-validation.jsonl:   0%|          | 0.00/4.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/18852 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/6538 [00:00<?, ? examples/s]

In [ ]:
dataset_train[0].keys()

dict_keys(['passage', 'question', 'answer', 'language', 'split'])

In [ ]:
dataset_train[0]

{'passage': 'Bahasa Parsi (/prn, -n/), juga dikenali dengan nama endonimnya Farsi (frsi (fsi) (dengar)), adalah salah satu bahasa Iran Barat dalam cabang Indo-Iran dari keluarga bahasa Indo-Eropah. Ia terutamanya dituturkan di Iran, Afghanistan (secara rasmi dikenali sebagai Dari sejak 1958), dan Tajikistan (secara rasmi dikenali sebagai Tajiki sejak zaman Soviet), dan beberapa kawasan lain yang secara historis merupakan masyarakat Parsi dan dianggap sebagai sebahagian daripada Iran Besar. Ia ditulis dalam abjad Parsi, varian yang diubah suai dari skrip Arab, yang sendiri berkembang dari abjad Aramia.',
 'question': 'adakah iran dan afghanistan bertutur dalam bahasa yang sama',
 'answer': 1,
 'language': 'Malay',
 'split': 'train'}

In [ ]:
# Define the create_prompt function
def create_prompt(sample):
    bos_token = "<s>"
    eos_token = "</s>"

    passage = sample['passage']
    summary = sample['question']
    answer = sample['answer']

    text_row = f"""[INST] Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca
              dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana kenyataan dalam
              soalan/kenyataan yang tidak disokong atau bercanggah dengan maklumat dalam dokumen).

              Anda perlu memilih antara dua pilihan berikut:
              - Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.
              - Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.

              Dokumen: {passage}
              Soalan/Kenyataan: {summary}

              Kembalikan jawapan dalam format JSON untuk pilihan yang diberikan. Sebagai contoh:
              - {{'consistency': 1}} atau {{'consistency': 0}}[/INST]"""

    answer_row = f"""{{"consistency": {answer}}}"""

    sample["prompt"] = bos_token + text_row
    sample["completion"] = answer_row + eos_token

    return sample

In [ ]:
dataset_instruct_format_train = dataset_train.map(create_prompt, remove_columns=['passage','question','answer','language', 'split'])

# print random sample
dataset_instruct_format_train[randint(0, len(dataset_instruct_format_train))]

Map:   0%|          | 0/18852 [00:00<?, ? examples/s]

{'prompt': "<s>[INST] Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca \n              dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana kenyataan dalam\n              soalan/kenyataan yang tidak disokong atau bercanggah dengan maklumat dalam dokumen).\n\n              Anda perlu memilih antara dua pilihan berikut:\n              - Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.\n              - Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.\n\n              Dokumen: Dalam 302 pilihan raya gubernur sejak tahun 1990, calon parti ketiga atau bebas telah memenangi sekurang-kurangnya 5.0% suara 49 kali (16%), sementara enam calon telah memenangi pilihan raya (2%). Parti ketiga atau gabenor bebas ya

### 2.0 Prepare the configuration for training the LLM
Making LLMs even more accessible with bitsandbytes, 4-bit quantization and QLoRA https://huggingface.co/blog/4bit-transformers-bitsandbytes

In [ ]:
model_id = "mesolitica/malaysian-mistral-7b-32k-instructions-v4"
new_model = "malaysian-mistral-qlora-7b-32k-instructions-llmasajudge" #set the name of the new model

In [ ]:
################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "bfloat16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True


################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule (constant a bit better than cosine)
lr_scheduler_type = "constant"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 25

# Log every X updates steps
logging_steps = 5

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = 32768

# Maximum batch size
dataset_batch_size = 32

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}
#device_map = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Load the base model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map=device_map
)

base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

# Load MitsralAi tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.79M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

In [ ]:
print(base_model)

In [ ]:
def find_all_linear_names(model):
    lora_module_names = set()
    for name, module in model.named_modules():
        if isinstance(module, bnb.nn.Linear4bit):
            names = name.split(".")
            lora_module_names.add(names[0] if len(names) == 1 else names[-1])

    if "lm_head" in lora_module_names:  # needed for 16-bit
        lora_module_names.remove("lm_head")
    return list(lora_module_names)

In [ ]:
# get lora target modules
modules = find_all_linear_names(base_model)

In [ ]:
print(modules)

Inference using base model only before fine tuning.

In [ ]:
eval_prompt = create_prompt(dataset_train[0])["prompt"]
print(eval_prompt)

<s>[INST] Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca 
              dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana kenyataan dalam
              soalan/kenyataan yang tidak disokong atau bercanggah dengan maklumat dalam dokumen).

              Anda perlu memilih antara dua pilihan berikut:
              - Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.
              - Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.

              Dokumen: Bahasa Parsi (/prn, -n/), juga dikenali dengan nama endonimnya Farsi (frsi (fsi) (dengar)), adalah salah satu bahasa Iran Barat dalam cabang Indo-Iran dari keluarga bahasa Indo-Eropah. Ia terutamanya dituturkan di Iran, Afghanistan (secara rasmi diken

In [ ]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

base_model.eval()
with torch.no_grad():
    print(tokenizer.decode(base_model.generate(**model_input, max_new_tokens=1024, pad_token_id=2)[0], skip_special_tokens=True))

[INST] Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca 
              dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana kenyataan dalam
              soalan/kenyataan yang tidak disokong atau bercanggah dengan maklumat dalam dokumen).

              Anda perlu memilih antara dua pilihan berikut:
              - Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.
              - Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.

              Dokumen: Bahasa Parsi (/prn, -n/), juga dikenali dengan nama endonimnya Farsi (frsi (fsi) (dengar)), adalah salah satu bahasa Iran Barat dalam cabang Indo-Iran dari keluarga bahasa Indo-Eropah. Ia terutamanya dituturkan di Iran, Afghanistan (secara rasmi dikenali

### 3.0 Train the LLM

Train on completions only https://huggingface.co/docs/trl/en/sft_trainer

In [ ]:
# Set LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    report_to="wandb",  # enable logging to W&B
    run_name=f'{new_model}',  # name of the W&B run (optional)
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    gradient_checkpointing=gradient_checkpointing,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=1000, # the total number of training steps to perform
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
)

In [ ]:
def formatting_prompts_func(example):
    output_texts = []
    for i in range(len(example['prompt'])):
        text = f"{example['prompt'][i]}\n\n ### Jawapan: {example['completion'][i]}"
        output_texts.append(text)
    return output_texts

response_template = "### Jawapan:"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
# Initialize the SFTTrainer for fine-tuning
trainer = SFTTrainer(
    model=base_model,
    train_dataset=dataset_instruct_format_train,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    dataset_batch_size=dataset_batch_size,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_batch_size, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:304: UserWarning: You passed a `dataset_batch_size` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/18852 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
# Start the training process
trainer.train()

wandb: Currently logged in as: adzhar-faiq. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


Step,Training Loss
5,0.264200
10,0.050900
15,0.106500
20,0.085000
25,0.087600
30,0.091600
35,0.067200
40,0.049300
45,0.078900
50,0.029000


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

KeyboardInterrupt: 

In [ ]:
wandb.finish()

train/epoch,0.10503
train/global_step,495
train/grad_norm,0.2055
train/learning_rate,0.0002
train/loss,0.064


In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)

In [ ]:
checkpoint = get_last_checkpoint('./results')
checkpoint

'./results/checkpoint-475'

In [ ]:
model = AutoModelForCausalLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
model.push_to_hub("loraadapter-malaysian-mistral-llmasajudge-v1", safe_serialization = True)
tokenizer.push_to_hub("loraadapter-malaysian-mistral-llmasajudge-v1", safe_serialization = True)

/usr/local/lib/python3.10/dist-packages/transformers/integrations/peft.py:397: FutureWarning: The `active_adapter` method is deprecated and will be removed in a future version.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/671M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/loraadapter-malaysian-mistral-llmasajudge-v1/commit/ae1bc602eef2ff980178af88884a726cf7600c54', commit_message='Upload tokenizer', commit_description='', oid='ae1bc602eef2ff980178af88884a726cf7600c54', pr_url=None, pr_revision=None, pr_num=None)

### 4.0 Merge the trained qlora into the base model

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    device_map=device_map
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
base_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

In [ ]:
merged_model= PeftModel.from_pretrained(base_model, new_model)

In [ ]:
merged_model

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralSdpaAttention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): l

In [ ]:
merged_model= merged_model.merge_and_unload()

In [ ]:
merged_model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
     

### 5.0 Upload model to HuggingFace

In [ ]:
merged_model.push_to_hub("malaysian-mistral-llmasajudge-v1", safe_serialization = True)

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/malaysian-mistral-llmasajudge-v1/commit/b3dc86d18c513fe946321d5b896c20f4d56f0815', commit_message='Upload MistralForCausalLM', commit_description='', oid='b3dc86d18c513fe946321d5b896c20f4d56f0815', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
tokenizer.push_to_hub("malaysian-mistral-llmasajudge-v1", safe_serialization = True)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/wanadzhar913/malaysian-mistral-llmasajudge-v1/commit/8176f8c20f3f73e221a1950b02c5325a6f3f3138', commit_message='Upload tokenizer', commit_description='', oid='8176f8c20f3f73e221a1950b02c5325a6f3f3138', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# test the model
eval_prompt = create_prompt(dataset_train[800])["prompt"]

In [ ]:
eval_prompt

'<s>[INST] Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca \n              dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana kenyataan dalam\n              soalan/kenyataan yang tidak disokong atau bercanggah dengan maklumat dalam dokumen).\n\n              Anda perlu memilih antara dua pilihan berikut:\n              - Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.\n              - Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.\n\n              Dokumen: Lebih terkenal, Harley-Davidson berusaha mendaftar sebagai tanda dagang khas `` chug \'\' enjin motosikal Harley-Davidson. Pada 1 Februari 1994, syarikat mengemukakan permohonannya dengan keterangan berikut: "Tanda terdiri daripada suara ekzo

In [ ]:
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

merged_model.eval()
with torch.no_grad():
    output = tokenizer.decode(merged_model.generate(**model_input, max_new_tokens=8, pad_token_id=2)[0], skip_special_tokens=True)

In [ ]:
output

'[INST] Anda adalah pakar dalam mengesan ketidakkonsistenan fakta dan halusinasi. Anda akan diberi satu dokumen dan satu soalan. Baca \n              dokumen dan soalan/kenyataan yang diberikan dengan teliti dan kenal pasti Ketidakkonsistenan Fakta (iaitu mana-mana kenyataan dalam\n              soalan/kenyataan yang tidak disokong atau bercanggah dengan maklumat dalam dokumen).\n\n              Anda perlu memilih antara dua pilihan berikut:\n              - Tidak Konsisten dengan Fakta: Jika mana-mana soalan/kenyataan tidak disokong, terjawab atau bercanggah dengan dokumen, labelkannya sebagai 0.\n              - Konsisten dengan Fakta: Jika semua soalan/kenyataan disokong/terjawab oleh dokumen, labelkannya sebagai 1.\n\n              Dokumen: Lebih terkenal, Harley-Davidson berusaha mendaftar sebagai tanda dagang khas `` chug \'\' enjin motosikal Harley-Davidson. Pada 1 Februari 1994, syarikat mengemukakan permohonannya dengan keterangan berikut: "Tanda terdiri daripada suara ekzos m